In [1]:
import fitsio
import numpy as np 
from desitarget.targetmask import desi_mask, bgs_mask, mws_mask, obsconditions

In [2]:
import matplotlib.pyplot as plt

# read targets

In [3]:
targets = fitsio.read('/global/project/projectdirs/desi/users/dsporta/forchang/bright_north_20_60_160_220_cut3.fits')

In [4]:
# unextinct fluxes
rflux = targets["FLUX_R"]/targets["MW_TRANSMISSION_R"]
zflux = targets["FLUX_Z"]/targets["MW_TRANSMISSION_Z"]
gflux = targets["FLUX_G"]/targets["MW_TRANSMISSION_G"]
w1flux = targets["FLUX_W1"]/targets["MW_TRANSMISSION_W1"]
rfibflux = targets["FIBERFLUX_R"]/targets["MW_TRANSMISSION_R"]

In [5]:
rmag = 22.5 - 2.5*np.log10(rflux.clip(1e-16))
zmag = 22.5 - 2.5*np.log10(zflux.clip(1e-16))
gmag = 22.5 - 2.5*np.log10(gflux.clip(1e-16))
w1mag = 22.5 - 2.5*np.log10(w1flux.clip(1e-16))
rfibmag = 22.5 - 2.5*np.log10(rfibflux.clip(1e-16))
color = (zmag  - w1mag) - 3/2.5 * (gmag - rmag) + 1.2

In [21]:
targets.dtype.names

('BRICKNAME',
 'RA',
 'DEC',
 'FLUX_G',
 'FLUX_R',
 'FLUX_Z',
 'FLUX_IVAR_G',
 'FLUX_IVAR_R',
 'FLUX_IVAR_Z',
 'MW_TRANSMISSION_G',
 'MW_TRANSMISSION_R',
 'MW_TRANSMISSION_Z',
 'FRACFLUX_G',
 'FRACFLUX_R',
 'FRACFLUX_Z',
 'FLUX_W1',
 'FLUX_W2',
 'FLUX_W3',
 'FLUX_W4',
 'FLUX_IVAR_W1',
 'FLUX_IVAR_W2',
 'FLUX_IVAR_W3',
 'FLUX_IVAR_W4',
 'MW_TRANSMISSION_W1',
 'MW_TRANSMISSION_W2',
 'MW_TRANSMISSION_W3',
 'MW_TRANSMISSION_W4',
 'FIBERFLUX_G',
 'FIBERFLUX_R',
 'FIBERFLUX_Z',
 'FIBERTOTFLUX_G',
 'FIBERTOTFLUX_R',
 'FIBERTOTFLUX_Z',
 'TARGETID',
 'SV1_DESI_TARGET',
 'SV1_BGS_TARGET',
 'SV1_MWS_TARGET',
 'SUBPRIORITY',
 'PRIORITY_INIT',
 'NUMOBS_INIT',
 'HPXPIXEL',
 'SV1_SCND_TARGET',
 'NUMOBS_MORE',
 'NUMOBS',
 'Z',
 'ZWARN',
 'TARGET_STATE',
 'TIMESTAMP',
 'VERSION',
 'PRIORITY',
 'OBSCONDITIONS')

# remove BGS targets that are not in MAIN OR FAINT

In [6]:
cut_main = rmag < 19.5
cut_faint = ((rmag > 19.5) & (rmag < 20.3) & ((rfibmag < 20.75) | ((rfibmag < 21.5) & (color > 0))))

In [7]:
old_bgs_main = (targets['SV1_BGS_TARGET'] & bgs_mask['BGS_BRIGHT']) != 0
old_bgs_faint = (targets['SV1_BGS_TARGET'] & bgs_mask['BGS_FAINT']) != 0 
old_bgs_faint_hip = (targets['SV1_BGS_TARGET'] & bgs_mask['BGS_FAINT_HIP']) != 0 

In [8]:
bgs_main_targnum = targets['SV1_BGS_TARGET'][old_bgs_main][0]
bgs_faint_targnum = targets['SV1_BGS_TARGET'][old_bgs_faint][0]
bgs_faint_hip_targnum = targets['SV1_BGS_TARGET'][old_bgs_faint_hip][0]

In [9]:
print(np.sum(old_bgs_main)/np.sum(targets['SV1_BGS_TARGET'].astype(bool)))
print(np.sum(old_bgs_faint)/np.sum(targets['SV1_BGS_TARGET'].astype(bool)))
print(np.sum(old_bgs_faint_hip)/np.sum(targets['SV1_BGS_TARGET'].astype(bool)))

0.38752153248135357
0.32715567821178443
0.025799645459364612


In [10]:
# remove all BGS targets outside of BGS main and faint
targets_trimmed = targets[
    (~targets['SV1_BGS_TARGET'].astype(bool)) | # not bgs targets
    ((targets['SV1_BGS_TARGET'].astype(bool) & cut_main) | (targets['SV1_BGS_TARGET'].astype(bool) & cut_faint))]

# assign updated `SV1_BGS_TARGET` values
- `MAIN` = r < 19.5
- `FAINT` = 66% of objects that pass (19.5 < r < 20.3 + RFIBER-COLOR cut)
- `FAINT_HIP` = 33% of objects that pass (19.5 < r < 20.3 + RFIBER-COLOR cut)

In [11]:
# unextinct fluxes
rflux = targets_trimmed["FLUX_R"]/targets_trimmed["MW_TRANSMISSION_R"]
zflux = targets_trimmed["FLUX_Z"]/targets_trimmed["MW_TRANSMISSION_Z"]
gflux = targets_trimmed["FLUX_G"]/targets_trimmed["MW_TRANSMISSION_G"]
w1flux = targets_trimmed["FLUX_W1"]/targets_trimmed["MW_TRANSMISSION_W1"]
rfibflux = targets_trimmed["FIBERFLUX_R"]/targets_trimmed["MW_TRANSMISSION_R"]

rmag = 22.5 - 2.5*np.log10(rflux.clip(1e-16))
zmag = 22.5 - 2.5*np.log10(zflux.clip(1e-16))
gmag = 22.5 - 2.5*np.log10(gflux.clip(1e-16))
w1mag = 22.5 - 2.5*np.log10(w1flux.clip(1e-16))
rfibmag = 22.5 - 2.5*np.log10(rfibflux.clip(1e-16))
color = (zmag  - w1mag) - 3/2.5 * (gmag - rmag) + 1.2

In [12]:
cut_main = rmag < 19.5
cut_faint = ((rmag > 19.5) & (rmag < 20.3) & ((rfibmag < 20.75) | ((rfibmag < 21.5) & (color > 0))))
rng_unif = np.random.uniform(size=len(cut_faint))

# Now lets change the FAINT_HIP percentage

In [13]:
for percentage in [33, 25, 20, 15]: 
    targets_trimmed['SV1_BGS_TARGET'][(targets_trimmed['SV1_BGS_TARGET'].astype(bool) & (cut_main | (cut_faint & (rng_unif < 0.01*percentage))))] = bgs_main_targnum
    targets_trimmed['SV1_BGS_TARGET'][(targets_trimmed['SV1_BGS_TARGET'].astype(bool) & cut_faint & (rng_unif > 0.01*percentage))] = bgs_faint_targnum
    
    targets_trimmed['PRIORITY_INIT'][(targets_trimmed['SV1_BGS_TARGET'].astype(bool) & (cut_main | (cut_faint & (rng_unif < 0.01*percentage))))] = 2100
    targets_trimmed['PRIORITY_INIT'][(targets_trimmed['SV1_BGS_TARGET'].astype(bool) & cut_faint & (rng_unif > 0.01*percentage))] = 2000
    
    targets_trimmed['PRIORITY'][(targets_trimmed['SV1_BGS_TARGET'].astype(bool) & (cut_main | (cut_faint & (rng_unif < 0.01*percentage))))] = 2100
    targets_trimmed['PRIORITY'][(targets_trimmed['SV1_BGS_TARGET'].astype(bool) & cut_faint & (rng_unif > 0.01*percentage))] = 2000

    new_bgs_main = (targets_trimmed['SV1_BGS_TARGET'] & bgs_mask['BGS_BRIGHT']) != 0
    new_bgs_faint = (targets_trimmed['SV1_BGS_TARGET'] & bgs_mask['BGS_FAINT']) != 0 
    
    print(np.sum(new_bgs_main)/np.sum(targets_trimmed['SV1_BGS_TARGET'].astype(bool)))
    print(np.sum(new_bgs_faint)/np.sum(targets_trimmed['SV1_BGS_TARGET'].astype(bool)))
    assert np.sum(new_bgs_main) + np.sum(new_bgs_faint) == np.sum(targets_trimmed['SV1_BGS_TARGET'].astype(bool))
    assert np.sum(new_bgs_main & new_bgs_faint) == 0
    print()
    fitsio.write('/global/project/projectdirs/desi/users/dsporta/forchang/bright_north_20_60_160_220_cut3.modified.hip%i.v2.fits' % percentage, targets_trimmed)

0.7173040371963942
0.28269596280360576

0.6837651468966289
0.31623485310337107

0.6627609379311152
0.33723906206888477

0.6415463172832522
0.3584536827167478

